# Model Experimentation
This notebook is designed to streamline experimentation with different models.

1. Selecting a model, dataset, tracker and hyperparameters
2. Training the model and evaluating both object detection and tracking performance
3. Saving the results in wandb
4. Storing the model and evaluation annotations in a folder
5. **Investigation** of the results, detection of failure cases and model improvement
6. **Iterating** over steps 1-5



In [3]:
from architectures import yolov8
from trackers import botsort
from data import yolo_dataset
import wandb
import json

ModuleNotFoundError: No module named 'image_processor'

> run `wandb login` in terminal before running this notebook

### Select a model, dataset, tracker and hyperparameters

In [2]:
architectures = {
  "yolov8": yolov8.YoloV8
}

trackers = {
  'botsort': botsort.BotSort
  }

dataset_mapping = {
 "yolov8": yolo_dataset.YoloDataset
  }

In [3]:
required_train_params = ["architecture", "data", "epochs", "batch_size", "img_size", "lr", "greyscale", "model_size", "model_path", "annotation_path", "patience"]
required_eval_params = ["conf_threshold", "eval_data", "iou_association_threshold", "tracker"]
dynamic = ["pretrained", "annotation_path"]

def construct_hyperparameters(model_name, **kwargs):
  """
  - train params: architecture, path, architecture, data, epochs, batch_size, img_size, train_mins, device
  - eval params: conf_tresh, device, eval_data
  """
  hyperparameters = {
    "model_name": model_name
  }

  model_pretrained = False

  with open("./assets/trained_models.json", "r") as file:
      trained_models = json.load(file)
  assert trained_models is not None
  
  if model_name in trained_models:
     model_pretrained = True
     # Train params are already known  
     model_train_params = trained_models[model_name]
     assert model_train_params is not None and [param in model_train_params for param in required_train_params]
     hyperparameters.update({param: model_train_params[param] for param in required_train_params})
     hyperparameters.update({param: kwargs[param] for param in required_eval_params})

  else:
    # TODO: account for tf models
    # TODO: allow for train_min, device s to be added afterwards
    model_path = "/vol/biomedic3/bglocker/ugproj2324/fv220/models/best.pt"
    assert len(kwargs) == len(required_train_params) + len(required_eval_params)
    assert [param in kwargs for param in required_train_params + required_eval_params]
    hyperparameters.update({**kwargs, "path": model_path})

  hyperparameters['pretrained'] = model_pretrained
  hyperparameters['annotations_path'] = "/".join(hyperparameters['model_path'].split("/")[:-1]) + "/annotations.csv"

  assert [param in hyperparameters for param in required_train_params + required_eval_params + dynamic]

  return hyperparameters

In [4]:
def model_experimentation(hyperparameters):
  run = wandb.init(project="SharkTrack", config=hyperparameters)

  try:
    tracker = trackers[hyperparameters['tracker']]()
    model = architectures[hyperparameters['architecture']](hyperparameters, tracker)

    data_dir = "/vol/biomedic3/bglocker/ugproj2324/fv220/datasets/images_raw/"
    dataset = dataset_mapping[hyperparameters['architecture']](data_dir, hyperparameters['data_sources'], hyperparameters['augmentations'])

    if not hyperparameters['pretrained']:
      train_time, device = model.train(dataset)
      wandb.log({"Training Time (m)": train_time, "Training Device": device})

    mota, motp, idf1, track_time, device =  model.evaluate()

    wandb.log({"MOTA": mota, "MOTP": motp, "IDF1": idf1, "Tracking Time (m)": track_time, "Tracking Device": device})
    # TODO: add image as well, test+time and test_device
     
  finally: 
    run.finish()
  


In [5]:
model_name = "yoloV8-medium-mvd2"
eval_params = {
  "conf_threshold": 0.2,
  "eval_data": "eval1",
  "iou_association_threshold": 0.5,
  "tracker": "botsort"
}

hyperparameters = construct_hyperparameters(model_name, **eval_params)
print(hyperparameters)

model_experimentation(hyperparameters)

{'model_name': 'yoloV8-medium-mvd2', 'architecture': 'yolov8', 'data': 'MVDv2', 'epochs': 50, 'batch_size': 16, 'img_size': 640, 'lr': 0.01, 'greyscale': False, 'model_size': 'm', 'model_path': '/vol/biomedic3/bglocker/ugproj2324/fv220/dev/shark_locator_tests/runs/detect/yolov8m_mvd2/best.pt', 'annotation_path': '/vol/biomedic3/bglocker/ugproj2324/fv220/dev/shark_locator_tests/runs/detect/yolov8m_mvd2/annotations.csv', 'conf_threshold': 0.2, 'eval_data': 'eval1', 'iou_association_threshold': 0.5, 'tracker': 'botsort', 'pretrained': True, 'annotations_path': '/vol/biomedic3/bglocker/ugproj2324/fv220/dev/shark_locator_tests/runs/detect/yolov8m_mvd2/annotations.csv'}


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: filippovarini (ml-driven-conservation). Use `wandb login --relogin` to force relogin


Initialised Model yoloV8-medium-mvd2 
Evaluating easy1


WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (1/598) /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/sharktrack_test/videos/easy1.mp4: 384x640 (no detections), 109.6ms
video 1/1 (2/598) /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/sharktrack_test/videos/easy1.mp4: 384x640 (no detections), 6.4ms
video 1/1 (3/598) /vol/biomedic3/bglocker/ugproj2324/fv220/datasets/sharktrack_test/videos/easy1.mp4: 384x640 1 shark, 6.2ms
video 1/

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


IDF1,▁
MOTA,▁
MOTP,▁
Tracking Time (m),▁
Device,NVIDIA GeForce RTX 4...
IDF1,0.58
MOTA,0.49
MOTP,0.41
Tracking Time (m),1.3
